In [1]:
import sys
from pathlib import Path

# Get the absolute path to the project root
notebook_path = Path('/Users/nirmal/Documents/np_research/ManGo_code/ManGo')
project_root = str(notebook_path)

# Add the project root to Python path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Now you can import your module
from src.magcal.gmed import gmedian

# Verify the import worked
print(f"Project root: {project_root}")
print(f"Available at: {gmedian.__module__}")

Project root: /Users/nirmal/Documents/np_research/ManGo_code/ManGo
Available at: src.magcal.gmed


In [2]:

# Sample data
import numpy as np
data = np.array([1.2, 2.3, 3.4, 4.5, 5.6])

# Using function directly
g_median_quant = gmedian(data, case='j')
print(f"i G-median: {g_median_quant:.4f}")

g_median_esti = gmedian(data, case='i')
print(f"j G-median: {g_median_esti:.4f}")

i G-median: 3.1887
j G-median: 3.2105


In [3]:
print(f'median {np.median(data)}')

median 3.4


In [4]:
print(f'mean {np.mean(data)}')

mean 3.4


In [ ]:
from src.magcal.gmodulus import gmodulus
from src.magcal.gvar import gvariance
from src.magcal.gacov import gautocovariance
from src.magcal import gcorrelation, gvariance, gmodulus, gautocovariance, gcovariance

In [6]:
data = np.array([1, 1, 1, 1.3, 1.1, 1, 1.4])

In [7]:
# Using function directly
g_median_quant = gmedian(data, case='j')
print(f"i G-median: {g_median_quant:.4f}")

g_median_esti = gmedian(data, case='i')
print(f"j G-median: {g_median_esti:.4f}")

i G-median: 1.1014
j G-median: 1.0790


In [8]:
Mi = gmodulus(data, case='i')
Mj = gmodulus(data, case='j')



In [9]:
print(f'Mi {Mi}, Mj {Mj}')

Mi 0.7586710175646536, Mj 1.0353479722893655


In [10]:
gvar_i = gvariance(data, 'i')
gvar_j = gvariance(data, 'j')

In [11]:
print(f'Gvar i:{gvar_i}, Gvar j:{gvar_j}')

Gvar i:6.2775181878091e-14, Gvar j:0.007967047408588992


In [12]:
# example of gvariance
data = np.array([1, 2, 3, 4, 5])
gvar_i = gvariance(data, 'i')
gvar_j = gvariance(data, 'j')
print(f'Gvar i:{gvar_i}, Gvar j:{gvar_j}')

Gvar i:2.0943247136528953e-12, Gvar j:-0.4570435138918648


In [13]:
# example of gcorrelation
data1 = np.array([1, 2, 3, 4, 5])
data2 = np.array([5, 4, 3, 2, 1])
gcor_i = gcorrelation(data1, data2, 'i')
gcor_j = gcorrelation(data1, data2, 'j')
print(f'Gcorrelation i:{gcor_i}, Gcorrelation j:{gcor_j}')

Gcorrelation i:-0.48811115231124125, Gcorrelation j:-2.436396290059431


In [15]:
# example of gautocovariance
data = np.array([1, 2, 3, 4, 5])
gautocov_i = gautocovariance(data, 'i')
gautocov_j = gautocovariance(data, 'j')
print(f'Gautocovariance i:{gautocov_i}, Gautocovariance j:{gautocov_j}')
# example of gcovariance
data1 = np.array([1, 2, 3, 4, 5])
data2 = np.array([5, 4, 3, 2, 1])
gcov_i = gcovariance(data1, data2, 'i')
gcov_j = gcovariance(data1, data2, 'j')
print(f'Gcovariance i:{gcov_i}, Gcovariance j:{gcov_j}')
# example of gmodulus
data = np.array([1, 2, 3, 4, 5])
gmod_i = gmodulus(data, 'i')
gmod_j = gmodulus(data, 'j')
print(f'Gmodulus i:{gmod_i}, Gmodulus j:{gmod_j}')
# example of gmedian
data = np.array([1, 2, 3, 4, 5])
gmed_i = gmedian(data, 'i')
gmed_j = gmedian(data, 'j')
print(f'Gmedian i:{gmed_i}, Gmedian j:{gmed_j}')
# example of gcorrelation
data1 = np.array([1, 2, 3, 4, 5])
data2 = np.array([5, 4, 3, 2, 1])
gcor_i = gcorrelation(data1, data2, 'i')
gcor_j = gcorrelation(data1, data2, 'j')
print(f'Gcorrelation i:{gcor_i}, Gcorrelation j:{gcor_j}')

Gautocovariance i:0.26767904840384404, Gautocovariance j:0.9855072394456723
Gcovariance i:-0.48811115231124125, Gcovariance j:-2.436396290059431
Gmodulus i:0.6563794353616111, Gmodulus j:1.7944203642047116
Gmedian i:2.824266155122618, Gmedian j:2.8084133025557407
Gcorrelation i:-0.48811115231124125, Gcorrelation j:-2.436396290059431
